In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install pydicom

     |████████████████████████████████| 35.5MB 89kB/s 


In [3]:
import os
from os.path import join as opjoin
base_path = '/content/drive/My Drive/DeepSEED-3D-ConvNets-for-Pulmonary-Nodule-Detection'
data_path = '/content/drive/My Drive/dsb2018_topcoders/data'
luna_path = opjoin(base_path, 'luna_detector')

In [ ]:
os.chdir(base_path)
!git pull

Already up to date.


In [ ]:
os.chdir(luna_path)
! CUDA_VISIBLE_DEVICES=0,1,2,3,4 python train_detector_se.py --test 0 -b 4 --save-dir 'test_results/baseline_128_cv_1' --mode theirs --epochs 105 --dataset_split 1 --resume 'baseline_128_cv_1/detector_056.ckpt'

creating model
FOCAL LOSS 2 0.5
created model
=> loading checkpoint 'baseline_128_cv_1/detector_056.ckpt'
using gpu 0
len lun train 800
len idcs  771
len(labels)  771
4
Epoch 057 (lr 0.00050)
Train:      tpr 92.80, tnr 99.28, total pos 1264, total neg 6968, time 1229.38
loss 0.0223, classify loss 0.0067, regress loss 0.0037, 0.0035, 0.0037, 0.0046
finsihed epoch 57
Epoch 058 (lr 0.00050)
Train:      tpr 93.74, tnr 99.45, total pos 1278, total neg 6968, time 1208.51
loss 0.0205, classify loss 0.0059, regress loss 0.0032, 0.0033, 0.0038, 0.0043
finsihed epoch 58
Epoch 059 (lr 0.00050)
Train:      tpr 94.43, tnr 99.51, total pos 1293, total neg 6968, time 1208.58
loss 0.0211, classify loss 0.0059, regress loss 0.0032, 0.0034, 0.0041, 0.0045
finsihed epoch 59
save model on epoch 59
Epoch 060 (lr 0.00050)
Train:      tpr 94.47, tnr 99.57, total pos 1266, total neg 6968, time 1210.34
loss 0.0197, classify loss 0.0052, regress loss 0.0038, 0.0031, 0.0034, 0.0042
finsihed epoch 60
Epoch 061 (l

In [ ]:
os.chdir(luna_path)
from roc_eval import SimpleTest

t = SimpleTest(path_to_model='baseline_5/detector_052.ckpt')
t.test_luna(4)

creating model
FOCAL LOSS 2 0.5
len idcs  83
len(labels)  83
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 128, 128, 128)
sample shape:  (1, 12

In [ ]:
os.chdir(luna_path)
import os
from importlib import import_module
from os.path import join as opjoin

import numpy as np
import torch
from tqdm import tqdm
from data_loader import LungNodule3Ddetector
from layers_se import *
from patient_data_loader import PatientDataLoader
from torch.utils.data import DataLoader
from abc import abstractmethod, ABC

from config_training import config as config_training


default_data_path = os.path.join('/content/drive/My Drive/DeepSEED-3D-ConvNets-for-Pulmonary-Nodule-Detection',
                                   config_training['preprocess_result_path'])

def init(data_path=None, path_to_model='', start=0, end=0, r_rand=0.9):
        data_path = default_data_path if data_path is None else data_path
        model = import_module('res18_se')
        print('creating model')
        config, net, loss, get_pbb = model.get_model()
        net = net.cuda()
        loss = loss.cuda()
        checkpoint = torch.load(opjoin(luna_path, 'test_results', path_to_model))
        net.load_state_dict(checkpoint['state_dict'])
        return net
        

In [ ]:
net  = init(path_to_model='baseline_5/detector_052.ckpt')

creating model
FOCAL LOSS 2 0.5


In [ ]:
data, target, coord = data.cuda(), target.cuda(), coord.cuda()
data = data.type(torch.cuda.FloatTensor)
coord = coord.type(torch.cuda.FloatTensor)

output = self.net(data, coord)
outputs.append(output.cpu().detach().numpy()[0])
targets.append(target)